In [1]:
import geemap, ee
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [ ]:
# 1. Vizualise Sentinel-2 images and clip around Pakistan.
# For visualization we’ll look at just the Red, Green and Blue channels

In [4]:
# get Pakistan boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','Pakistan')).geometry()

# Sentinel-2 image filtered on 2019 and on Pakistan
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate("2019-01-01","2019-12-31").filterBounds(aoi).median().divide(10000)

rgb = ['B4','B3','B2']

# set some thresholds
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}


# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 5)
map1.addLayer(se2.clip(aoi), rgbViz, "S2")

map1.addLayerControl()
map1

Map(center=[29.35382169482654, 68.69360662933832], controls=(WidgetControl(options=['position', 'transparent_b…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [ ]:
# 2. Remove cloud cover by creating a cloud mask to clear the image up using Sentinel-2's QA band

In [5]:
def se2mask(image):
    quality_band = image.select('QA60')
    
    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    
    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    
    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask).divide(10000)

se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2019-01-01","2019-12-31").filterBounds(aoi).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).median()

# initialize our map
map2 = geemap.Map()
map2.centerObject(aoi, 5)
map2.addLayer(se2.clip(aoi), rgbViz, "S2")

map2.addLayerControl()
map2

Map(center=[29.35382169482654, 68.69360662933832], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# 3. Using Sentinel-2 data

# The dataset we are particularly interested in is the GHSL “Settlement Grid” layer
# First, there is one band with four “degrees of urbanization”:
# Inhabited areas
# Rural grid cells
# Low Density Clusters (towns and cities)
# High Density Clusters (cities)

# we are interested in the change of the Low and High Density clusters (“built up”) relative to everything else, 
# so we will classify any pixel with values in [3,4] as “built up” and assign this 1 or not and assign it 0.

In [ ]:
# Visualise the built-up/ non buit-up places

In [12]:
ghsl = ee.ImageCollection('JRC/GHSL/P2016/SMOD_POP_GLOBE_V1').filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('smod_code').median();

# create a boolean mask setting anything equaling 2 (low density) or 3 (high density) as True
# this will actually be our binary label layer
ghslbinary = ghsl.gte(2)

ghslVis= {"min":0.0, "max":3.0,"palette":['000000', '448564', '70daa4', 'ffffff']}
ghslbiVis= {"palette":['000000', 'ffffff']}

map3 = geemap.Map()
map3.centerObject(aoi,5)
map3.addLayer(ghsl.clip(aoi), ghslVis, 'Degree of Urbanization')
map3.addLayer(ghslbinary.clip(aoi), ghslbiVis, 'Built up')
map3

Map(center=[29.35382169482654, 68.69360662933832], controls=(WidgetControl(options=['position', 'transparent_b…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [13]:
# Moving focus to sth province which contains Islamabad, the capital city:
# Integrating the feature sources (Sentinel-2 and VIIRS-DNB) as well as the GHSL data as layers

In [21]:
# our Region Of Interest is the Province of ???? TODO get province!!
roi = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Islamabad District')).geometry()
    
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2019-01-01","2019-12-31").filterBounds(roi).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).median().clip(roi)

viirs = ee.Image(ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2019-01-01","2019-12-31").filterBounds(roi).median().select('avg_rad').clip(roi))

ghsl = ee.ImageCollection('JRC/GHSL/P2016/SMOD_POP_GLOBE_V1').filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('smod_code').median().clip(roi)

ghsl = ghsl.gte(2)

ghslVis= {"palette":['000000', 'ffffff']}
se2Vis = {"min":0.0, "max":0.3,"bands": ['B4','B3','B2']}

# initialize our map
map4 = geemap.Map()
map4.centerObject(roi, 9)
map4.addLayer(se2, se2Vis, "S2")
map4.addLayer(viirs, {}, "VIIRS-DNB", opacity=0.5)
map4.addLayer(ghsl, ghslVis, "GHSL", opacity=0.25)
map4.addLayerControl()
map4

Map(center=[33.67077119814098, 73.11761745663053], controls=(WidgetControl(options=['position', 'transparent_b…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [ ]:
# 4. Data exploration (look at changes in these underlying datasets to spot any possible biases or inconsisten
# cies (such as unexpected spikes or drops in the data over time or interesting spatial distributions))

In [ ]:
# sneak peak at the VIIRS to compare late 2015 with 2019.

# Note that we only look at the latter half of the year, because we do not have Sentinel-2 1-C data prior to July 
# 2015 and we want to compare the same months of the year in 2015 as 2019 for both data sources.



In [23]:
viirs2015 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2015-07-01","2015-12-31").filterBounds(roi).median().select('avg_rad').clip(roi)
viirs2019 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2019-07-01","2019-12-31").filterBounds(roi).median().select('avg_rad').clip(roi)

viirs_15_tile = geemap.ee_tile_layer(viirs2015, {}, 'Jul-Dec 2015', opacity=0.75)
viirs_19_tile = geemap.ee_tile_layer(viirs2019, {}, 'Jul-Dec 2019', opacity=0.75)

# initialize our map
map5 = geemap.Map()
map5.centerObject(roi, 9)
map5.split_map(left_layer=viirs_15_tile, right_layer=viirs_19_tile)
map5.addLayerControl()
map5

Map(center=[33.67077119814098, 73.11761745663053], controls=(ZoomControl(options=['position', 'zoom_in_text', …

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [ ]:
# Let’s try cleaning the data a bit to get a clearer signal to noise ratio. 
# We’ll subtract the mean and divide by the standard deviation (also known as “standardizing” or “scaling” the data).

In [24]:
mu15 = viirs2015.reduceRegion(reducer=ee.Reducer.mean(),scale=500)
std15 = viirs2015.reduceRegion(reducer=ee.Reducer.stdDev(),scale=500)
mu19 = viirs2019.reduceRegion(reducer=ee.Reducer.mean(),scale=500)
std19 = viirs2019.reduceRegion(reducer=ee.Reducer.stdDev(),scale=500)

# we'll cast these to native ee Numbers using the ee.Number constructor
mu15 = ee.Number(mu15.get('avg_rad'))
std15 = ee.Number(std15.get('avg_rad'))
mu19 = ee.Number(mu19.get('avg_rad'))
std19 = ee.Number(std19.get('avg_rad'))

viirs2015 = viirs2015.subtract(mu15).divide(std19)
viirs2019 = viirs2019.subtract(mu15).divide(std19)

viirs_15_tile = geemap.ee_tile_layer(viirs2015, {}, 'Jul-Dec 2015', opacity=0.75)
viirs_19_tile = geemap.ee_tile_layer(viirs2019, {}, 'Jul-Dec 2019', opacity=0.75)

# initialize our map
map6 = geemap.Map()
map6.centerObject(roi, 9)
map6.split_map(left_layer=viirs_15_tile, right_layer=viirs_19_tile)
map6.addLayerControl()
map6

Map(center=[33.67077119814098, 73.11761745663053], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
# we lost some information, which is potentially important if we want to pick up dimly lit 
# areas that are transitioning from rural to low density.

# Aside from this, there are no major anormalies with the data such as if there were some more lit places in 2015
# compared to 2019. If there were any, we'd need to clean up the data. This is a very important aspect of data 
#  exploration: finding ways we may need to adjust the data. If our classifier fails to perform well, we may need 
# to experiment with cleaning the data like this prior to training.

# Sometimes machine learning algorithms that are too senstive will “learn” noise too well and then perform badly 
# on novel data, so cleaning data, which is by definition removing information, may help your classifier be more robust.


In [ ]:
# Data bias and the importance of combinig different data sources. 

# GHSL has someinformation for that layer derived from remote sensing, including Sentinel-2 itself. 
# This means that is the classifier is using Sentinel-2 data alone to classify data for built-up areas,
# it may already have bias. 

# In other words, your classifier might “perform well” on your labeled data…but what if the labels 
# themselves are off in the same way your input data is?

# This is a good reminder why it is helpful to integrate multiple independent data sources 
# (such as Sentinel-2 MSI daytime imagery) into analysis. The Sentinel-2 may present a bias with GHSL and 
#  be less sensitive to time dynamics, so we can balance it with nighttime lights. But it also has a higher 
#  signal to noise ratio spatially and more spectral information, so it counters the noise we see in nighttime lights alone.



In [ ]:
# 5. Merging different data sources

# We are ultimately looking to classify land cover using both VIIRS-DNB and Sentinel-2, so we should 
# merge them into a single file to make it easier to pass to our classifier.


In [ ]:
# Spatial Resolution
# VIIRS-DNB has a spatial resolution of about 750 meters. For Sentinel-2 MSI Level1-C data it ranges from 10
# to 60 meters.GHSL has a resolution of 1 km. The best apporach is to resample the data. Options include:
# - downsample the higher-resolution data
# - disaggregate the lower resolution data to a smaller pixel size (Generative Adversarial Networks
# - re-sample all images to some separate resolution (say…500 meters)

# Solution for today: re-sample the data to 1000 meter resolution.


# Temporal resolution (time)
# GHSL data is a rather static dataset generally time-stamped 2015 (although the underlying data is somewhat varied).
# Solution: Rreate a dataset that is a single representation (in time) of all the data in 2015, i.e. a single composite image.
# (July through December due to Sentinel limitation of available data)

# Note:
# For our inference data set (2016 to 2019) we will predict built-up land for each year, but we could do this per month 
# if we wanted to given our VIIRS-DNB and Sentinel-2 frequency. It would give us a denser time series but is more work to calculate.



In [48]:
# Combine the VIIRS-DNB radiance band with the Sentinel-2 bands into a single Image object. For the Sentinel-2 bands,
# we’ll choose the visible and near infra-red bands.

se2bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A'] #"Image.select: Pattern 'B2' did not match any bands.

# TMP ROI CHANGE TO GET RID OF BANDS ERROR!!!
roi = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Bagmati')).geometry()

se2_2015 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2015-07-01","2015-12-31").filterBounds(roi).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).median().clip(roi).select(se2bands)

viirs2015 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2015-07-01","2015-12-31").filterBounds(roi).median().select('avg_rad').clip(roi)

fused = se2_2015.addBands(viirs2015)

In [ ]:
# For the GHSL, we’ll want to extract points cooresponding with our “built-up” class that we can overlay over our feature data.

# We’re choosing 1000 meters as our overall scaling factor for all data, a sample rate that matches the resolution of the GHSL native resolution.

In [49]:
# Note in the out below, the “property” field name from GHSL: “smod_code”. We’ll use this to assign 
# the training data labels (1 for built up, 0 for not built up after we binarized the “degrees of urbanization”).
ghslpoints = ghsl.sample(**{"region":roi, "scale":1000,"seed":0,'geometries':True})
ghslpoints.size().getInfo()
ghslpoints.first().getInfo()

In [53]:
# overlay these labels on the fused training image:

# get list of all bands we're using in Sentinel-2 and VIIRS-DNB
trainingbands = se2bands + ['avg_rad']

training = fused.select(trainingbands).sampleRegions(collection=ghslpoints,
                                                     properties=['smod_code'],
                                                     scale=1000)

# glance at the underlying data…a view of the first record or observation (i.e. pixel):
training.first().getInfo()

In [52]:
# Stats for smod_code our binary label for built up (1) or not (0)
training.aggregate_stats('smod_code').getInfo()

{'max': None,
 'mean': 0,
 'min': None,
 'sample_sd': 0,
 'sample_var': 0,
 'sum': 0,
 'sum_sq': 0,
 'total_count': 0,
 'total_sd': 0,
 'total_var': 0,
 'valid_count': 0,
 'weight_sum': 0,
 'weighted_sum': 0}

In [ ]:
# (for Nepal data): Imbalanced class: We notice the mean value for our label is 0.18, which indicates 
# a fairly high ratio of non built up land (0) relative to built-up land (1). 
# While this is expected behavior in such a real world dataset, class imbalance can be a challenge for classifiers.
        
        
        